In [4]:
import numpy as np

# Load Data
train_data = np.loadtxt("boston_train.csv", delimiter=",")
test_data = np.loadtxt("boston_test.csv", delimiter=",")

# Separate features (X) and target (y)
train_X = train_data[:, :-1]
train_y = train_data[:, -1]
test_X = test_data  # Test data doesn't have target values

# Standardization (Feature Scaling)
def standardize(X):
    mean = np.mean(X, axis=0)
    std = np.std(X, axis=0)
    std[std == 0] = 1  # Avoid division by zero
    return (X - mean) / std, mean, std

train_X, mean, std = standardize(train_X)
test_X = (test_X - mean) / std  # Apply same mean & std to test data

# Add bias term (column of ones) to train and test sets
train_X = np.c_[np.ones(train_X.shape[0]), train_X]
test_X = np.c_[np.ones(test_X.shape[0]), test_X]

# Compute Cost Function (Mean Squared Error)
def compute_cost(X, y, m):
    N = len(y)
    predictions = np.dot(X, m)
    cost = (1 / (2 * N)) * np.sum((predictions - y) ** 2)
    return cost

# **Optimized Gradient Descent with Momentum**
def gradient_descent(X, y, learning_rate=0.01, num_iterations=5000, tolerance=1e-6, beta=0.9):
    m = np.zeros(X.shape[1])  # Initialize weights
    prev_cost = float('inf')  # Set initial cost to a large value
    cost_history = []  # Store cost for each iteration
    v = np.zeros_like(m)  # Initialize momentum term

    for i in range(num_iterations):
        predictions = np.dot(X, m)
        error = predictions - y
        gradient = (1 / len(y)) * np.dot(X.T, error)
        
        # **Momentum Update**
        v = beta * v + (1 - beta) * gradient
        m -= learning_rate * v  # Update weights

        # Calculate cost
        cost = compute_cost(X, y, m)
        cost_history.append(cost)

        # **Adaptive Learning Rate**
        if i > 0 and cost > cost_history[-2]:  
            learning_rate *= 0.5  # Reduce learning rate if cost increases

        # **Early Stopping**
        if abs(prev_cost - cost) < tolerance:
            print(f"Stopping early at iteration {i+1}, Cost: {cost:.5f}")
            break

        prev_cost = cost  # Update previous cost

    return m, cost_history

# Train Model
learning_rate = 0.1  # Increased initial learning rate
num_iterations = 10000  # More iterations for better convergence
m, cost_history = gradient_descent(train_X, train_y, learning_rate, num_iterations)

print("Trained Coefficients (m):", m)
print("Final Cost:", cost_history[-1])

# Make Predictions
predictions = np.dot(test_X, m)

# Save predictions to CSV file
np.savetxt("boston_pred_2.csv", predictions, fmt="%.5f")  # Save with 5 decimal places
print("Predictions saved to 'boston_predictions.csv'.")


Trained Coefficients (m): [22.60949868 -0.98535169  0.70354942 -0.09130155  0.82964608 -2.14960873
  2.36705603  0.11052039 -3.03029411  2.34902299 -1.47651591 -2.23979219
  0.5925507  -4.31880755]
Final Cost: 11.737187870437172
Predictions saved to 'boston_predictions.csv'.
